In [15]:
from config import read_config
from dataset import get_loaders


configs = read_config()
_, _, _, _, train_set, _ = get_loaders(configs, device="cpu")
train_set.alpha=0

sad_sample = train_set[0]
neutral_sample = train_set[1]

In [19]:
sad_sample.keys()

dict_keys(['id', 'speaker', 'text', 'raw_text', 'mel', 'pitch', 'energy', 'duration', 'emotion2id'])

In [30]:
print(sad_sample['id'], "|", 
      sad_sample['emotion2id'], "|",
      sad_sample['raw_text'], "|",
)

print(neutral_sample['id'], "|", 
      neutral_sample['emotion2id'], "|",
      neutral_sample['raw_text'], "|",
)
import torch
mel = torch.from_numpy(neutral_sample['mel']).T
# mel to audio:
#
# import librosa
# S = librosa.feature.inverse.mel_to_stft(mel)
# y = librosa.griffinlim(S)

# from IPython.display import Audio
# Audio(y, rate=16000)

STFT = Audio.stft.TacotronSTFT(
            1024, # config["preprocessing"]["stft"]["filter_length"],
            256, # config["preprocessing"]["stft"]["hop_length"],
            1024, # config["preprocessing"]["stft"]["win_length"],
            80, # config["preprocessing"]["mel"]["n_mel_channels"],
            22050, # config["preprocessing"]["audio"]["sampling_rate"],
            0, # config["preprocessing"]["mel"]["mel_fmin"],
            8000 # config["preprocessing"]["mel"]["mel_fmax"],
        )
STFT._stft_fn = STFT.stft_fn # hotfix
import audio as Audio
Audio.tools.inv_mel_spec(mel, "neu.wav", STFT)

005141 | 3 | tôi bắt có làm việc nhà à |
008270 | 0 | tôi không thể giúp cô vì anh ta ý cô là sao |


In [41]:
from dataset import mixup_data

emo_mel = sad_sample['mel']
neu_mel = neutral_sample['mel']

min_len = min(emo_mel.shape[0], neu_mel.shape[0])
emo_mel = emo_mel[:min_len]
neu_mel = neu_mel[:min_len]

mixup_mel, lam = mixup_data(emo_mel, neu_mel, lam=0.6)
mixup_mel = torch.from_numpy(mixup_mel).T
Audio.tools.inv_mel_spec(mixup_mel, "mix-lamb=0.6.wav", STFT)